<a href="https://colab.research.google.com/github/iagomauricioo/data_science/blob/main/mapa_marechal_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import folium
import pandas as pd
from geopy.distance import geodesic
from IPython.display import HTML

In [3]:
barra_sao_miguel_coords = [-9.8379, -35.8979]
marechal_deodoro_coords = [-9.7092, -35.8964]

In [4]:
center_lat = (barra_sao_miguel_coords[0] + marechal_deodoro_coords[0]) / 2
center_lon = (barra_sao_miguel_coords[1] + marechal_deodoro_coords[1]) / 2
center_coords = [center_lat, center_lon]

In [5]:
m = folium.Map(location=center_coords, zoom_start=12)

In [6]:
folium.Marker(location=barra_sao_miguel_coords, popup='Barra de São Miguel', icon=folium.Icon(color='blue')).add_to(m)
folium.Marker(location=marechal_deodoro_coords, popup='Marechal Deodoro', icon=folium.Icon(color='green')).add_to(m)

In [7]:
radius = 10000  # 10 km em metros
folium.Circle(location=center_coords, radius=radius, color='red', fill=True, fill_color='red', fill_opacity=0.2).add_to(m)

In [8]:
m

# Área onde será realizado o mapa de calor em breve.

In [15]:
arquivo_excel = '/content/drive/MyDrive/coordenadas.xlsx'
df = pd.read_excel(arquivo_excel)
df.dropna(inplace=True)
df

,tipo,cidade,coordenadas
0,Mercado,Barra de São Miguel,"-9.840822033037991, -35.905050152891015"
1,Mercado,Barra de São Miguel,"-9.841170945190338, -35.904413953028474"
2,Mercado,Barra de São Miguel,"-9.843521890587615, -35.903697672912195"
3,Mercado,Barra de São Miguel,"-9.84291147460505, -35.9020740760202"
4,Mercado,Barra de São Miguel,"-9.842722965624427, -35.90114136570571"
5,Mercado,Barra de São Miguel,"-9.842063183347923, -35.89884546306096"
6,Mercado,Barra de São Miguel,"-9.841615473193242, -35.89831931878099"
7,Mercado,Barra de São Miguel,"-9.841662600606524, -35.89793666839556"
8,Mercado,Barra de São Miguel,"-9.841379836025952, -35.897171367624686"
9,Mercado,Barra de São Miguel,"-9.840437285675714, -35.89698004243198"


In [16]:
coordenadas = df['coordenadas']
coordenadas = coordenadas.str.split(',', expand=True)
coordenadas[0] = coordenadas[0].astype(float)
coordenadas[1] = coordenadas[1].astype(float)
coordenadas.head(3)

,0,1
0,-9.840822,-35.905050
1,-9.841171,-35.904414
2,-9.843522,-35.903698


In [17]:
for _, row in coordenadas.iterrows():
    m.add_child(
        folium.Marker(
            location=[row[0], row[1]],
            popup="Tipo de estabelecimento: Mercado",
            icon=folium.Icon(color='purple')
        )
    )

# Comparação
Para efeitos de comparação, na primeira imagem abaixo podemos ver uma imagem diretamente do Google Maps. Já na segunda imagem, vemos um mapa feito em python com os locais traçados iguais ao google maps. (Apenas para vermos o quão parecido está para ter uma noção de margem de erros).

In [18]:
iframe_code = """
<iframe src="https://www.google.com/maps/embed?pb=!1m16!1m12!1m3!1d31449.56301026032!2d-35.90486405726632!3d-9.833951979960371!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!2m1!1ssupermercado!5e0!3m2!1spt-BR!2sbr!4v1718769099993!5m2!1spt-BR!2sbr" width="600" height="450" style="border:0;" allowfullscreen="" loading="lazy" referrerpolicy="no-referrer-when-downgrade"></iframe>
"""

HTML(iframe_code)


## Na imagem acima, vemos a quantidade de supermercados na Barra de São Miguel.

In [19]:
m